# Sandbox
For testing and developing new Cyber Security Assessment tools in an interactive and persistent development environment.

In [ ]:
import itertools
import json
import random
import math
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.stats.distributions as distr
import seaborn as sns
from pathlib import Path as p

from cyber import Defence, Vulnerability, CommmonDefences, CyberComponent
from tree import TreeNode, Link
from comm_network import Aggregator, Device, CommNetwork
from comm_network import LevelOfChildren
from attackers import RandomAttacker

## Procedural Generation
### Abstract Tree
Consists of Devices and Aggregators. 
* Aggregators (internal nodes) require a **Hard** amount of effort to compromise and have a 50% chance of being compromised if the necessary effort is spent
* Devices (leaf nodes) require an **Easy** amount of effort to compromise and also have a 50% chance of being compromised if the necesssary effort is spent
* Control Center (root node) is **Very Hard** to compromise

Controllable parameters include:
* Number of devices (leaf nodes)
* Number of Entrypoints (points where cyberattacks can originate)
* Level of Redundancy (number of children per parent node)
* Random deviation in Redundancy
* Sibling to Sibling communication (lateral edges between nodes on the same level)

In [ ]:
# seed = np.random.randint(low=0, high=52600)
seed = 27194
np.random.seed(seed); random.seed(seed)
print(f"Seed: {np.random.get_state()[1][0]}")
pcn = CommNetwork(n_devices=30, n_entrypoints=1, children_per_parent=28, child_no_deviation=1, 
                  network_specs=p.cwd() / "SmartMeterNetworkSpecifications.json",
                  enable_sibling_to_sibling_comm=True)
print(CommNetwork.show_tree(pcn.root))
print(f"Number of Components: {pcn.n_components}")

## Monte Carlo
Build an approximate profile of the network's cyber security by launching many cyber attacks. The higher N_ATTACKS the more precise the resulting distribution is, however this comes at the cost of increased computation time.
The more nodes are compromised, the more successful the attack.

### Active Graph Only
Only perform Monte Carlo simulation on the currently active network.

In [ ]:
N_ATTACKS = 1000
BUDGET = 52
compromised_array = np.zeros(shape=N_ATTACKS, dtype=np.int16)
effort_array = np.zeros(shape=N_ATTACKS, dtype=np.float32)
for attack_no in range(N_ATTACKS):
    attacker = RandomAttacker(budget=BUDGET, verbose=False)
    nodes_compromised, total_effort_spent = attacker.attack_network(pcn)
    compromised_array[attack_no] = len([n for n in nodes_compromised if isinstance(n, Device)])
    effort_array[attack_no] = total_effort_spent
    pcn.reset()

In [ ]:
# Histogram
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8,6))
fig.suptitle(f"Attacker: {attacker.__name__}, Budget: {BUDGET}\nNetwork Size: {pcn.n_components}, No. of Devices: {pcn.n_devices}, No. of Entrypoints: {pcn.n_entrypoints}", 
             y=-0.05, fontsize="medium", ma="center")
sns.histplot(compromised_array, discrete=True, stat="probability", ax=axes[0])
axes[0].set(xticks=np.arange(0, len(pcn.graph.nodes())), xlabel="No. of Devices Compromised")
sns.histplot(effort_array, binwidth=1, ax=axes[1])
axes[1].set(xlabel="Effort Spent")
plt.tight_layout()
plt.show()

### Varied Parameter
Perform monte carlo simulation while varying particular parameter, such as the level of redundancy in the network. 

In [ ]:
import os
import multiprocess as mp
N_ATTACKS = 10000
N_DEVICES = 30
BUDGET = 52
SPEC = p.cwd() / "SmartMeterNetworkSpecifications.json"
SEED = np.random.randint(low=0, high=52600)
N_ENTRYPOINTS = 1 # Total budget is multiplied by this!
MIN_CHILDREN = 2
MAX_CHILDREN = N_DEVICES
CHILD_NO_STEP = 2
CHILD_NO_DEVIATION = 1
no_of_children = np.arange(MIN_CHILDREN, MAX_CHILDREN, CHILD_NO_STEP)
network_specs = dict(n_devices=N_DEVICES,
                     n_entrypoints=N_ENTRYPOINTS,
                     network_specs=SPEC,
                     child_no_deviation=CHILD_NO_DEVIATION,
                     enable_sibling_to_sibling_comm=True)

print(f"Seed: {SEED}")
np.random.seed(SEED)

compromised_array = np.zeros(shape=(N_ATTACKS, len(no_of_children)), dtype=np.int16)
effort_array = np.zeros(shape=(N_ATTACKS, len(no_of_children)), dtype=np.float32)
print(f"CPU Thread Count: {mp.cpu_count()-2}")

def monte_carlo(process_idx, seed, n_attacks, budget, **network_kwargs):
    import os
    import numpy as np
    from comm_network import CommNetwork, Device
    from attackers import RandomAttacker
    
    # Procedurally generate a communication network with specific redundancy
    np.random.seed(seed)
    pcn = CommNetwork(**network_kwargs)

    # Store effort and no. of devices compromised
    compromised_array = np.zeros(shape=n_attacks, dtype=np.int16)
    effort_array = np.zeros(shape=n_attacks, dtype=np.float32)

    for attack_no in range(n_attacks):
        attacker = RandomAttacker(budget=budget, verbose=False)
        nodes_compromised, total_effort_spent = attacker.attack_network(pcn)
        compromised_array[attack_no] = len([n for n in nodes_compromised if isinstance(n, Device)])
        effort_array[attack_no] = total_effort_spent
        # Entrypoint changes with each attack (i.e. same network different entrypoint)
        pcn.reset() 
    return process_idx, compromised_array, effort_array

with mp.Pool(processes=len(no_of_children)) as pool:
    results = []
    for i, children_per_parent in enumerate(no_of_children):
        print("Children per parent:", children_per_parent)
        kwds = {**network_specs, **dict(children_per_parent=children_per_parent)}
        results.append(
            pool.apply_async(monte_carlo, args=[i, SEED, N_ATTACKS, BUDGET], kwds=kwds)
        )
    
    for result in results:
        process_idx, compromises, efforts = result.get()
        compromised_array[:, process_idx] = compromises
        effort_array[:, process_idx] = efforts

In [ ]:
import pandas as pd
df = pd.DataFrame(compromised_array, columns=no_of_children)
df = df.melt(var_name='Children')

display(df)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
sns.histplot(df, x="value", hue="Children", discrete=True, stat="probability", common_norm=False, ax=ax)
sns.move_legend(ax, "upper right", ncols=4, title="Children per Parent")
ax.set(xlabel="No. of Devices Compromised")
plt.show()

## Visualization
Plot the structure of the communication network. 

In [ ]:
from matplotlib.patches import Patch

def lighten_color(color, amount=0.5):
    """
    Credit: Ian Hincks (https://stackoverflow.com/questions/37765197/darken-or-lighten-a-color-in-matplotlib)
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

def hierarchy_pos(G:nx.DiGraph, root:TreeNode, width:float=1., vert_gap:float=0.2, vert_loc:float=0, xcenter:float=0.5):

    '''
    Credit: Joel (https://stackoverflow.com/a/29597209/2966723) 
    Licensed under CC Attribution-Share Alike 
    
    
    If the graph is a tree this will return the positions to plot this in a 
    hierarchical layout.
    
    G (networkx.DiGraph): Graph (must be a tree)
    root (Node): Root node of current graph
    width (float): Horizontal space allocated for this branch - avoids overlap with other branches. Defaults to 1.0
    vert_gap (float): Gap between levels of hierarchy. Defaults to 0.2
    vert_loc (float): Vertical location of root. Defaults to 0.0
    xcenter (float): Horizontal location of root. Defaults to 0.5
    '''
    # if not nx.is_tree(G):
    #     raise TypeError('cannot use hierarchy_pos on a graph that is not a tree')

    if root is None:
        if isinstance(G, nx.DiGraph):
            root = next(iter(nx.topological_sort(G)))  #allows back compatibility with nx version 1.11
        else:
            root = np.random.choice(list(G.nodes))

    def _hierarchy_pos(G, root, width=1., vert_gap = 0.2, vert_loc = 0, xcenter = 0.5, pos = None, parent = None):
        '''
        see hierarchy_pos docstring for most arguments

        pos: a dict saying where all nodes go if they have been assigned
        parent: parent of this branch. - only affects it if non-directed

        '''
    
        if pos is None:
            pos = {root:(xcenter,vert_loc)}
        else:
            pos[root] = (xcenter, vert_loc)
        children = root.children # list(G.neighbors(root))
        if not isinstance(G, nx.DiGraph) and parent is not None:
            for child in children:
                child.remove_parents(parent)  
        if len(children) !=0:
            dx = width/len(children) 
            nextx = xcenter - width/2 - dx/2
            for child in sorted(children, key=lambda child:child.id):
                nextx += dx
                pos = _hierarchy_pos(G,child, width = dx, vert_gap = vert_gap, 
                                    vert_loc = vert_loc-vert_gap, xcenter=nextx,
                                    pos=pos, parent = root)
        return pos

            
    return _hierarchy_pos(G, root, width, vert_gap, vert_loc, xcenter)

def plot_communication_network(network:CommNetwork, palette:str="tab10"):
    """
    Plots a tree-like and spring layout of the given communication network.
    The visualization shows:
    * Entrypoints for possible cyberattacks
    * Different component types present in the network
    * Components that have been compromised

    Args:
        network (CommNetwork): A specific communication network
    """
    node_color_mask = np.full(network.graph.number_of_nodes(), fill_value="#1f78b4", dtype=object)
    node_edge_color_mask = np.full(network.graph.number_of_nodes(), fill_value="#000000", dtype=object)
    edge_color_mask = np.full(network.graph.number_of_edges(), fill_value="#000000", dtype=object)

    # sns.color_palette("hls", 12)
    node_types = set(node.name for node in network.graph.nodes())
    color_lookup = {k:v for k,v in zip(node_types,
                        sns.color_palette(palette, n_colors=len(node_types)))}

    # Custom Legend
    legend_map = {}
    for i, node in enumerate(network.graph.nodes()):
        color = color_lookup[node.name]
        name = node.name
        # Lighten the color of nodes which are Entrypoints for potential cyberattacks
        if node.is_accessible:
            node_color_mask[i] = lighten_color(color, amount=0.6)
            name += " (entrypoint)"
        else:
            node_color_mask[i] = color

        if node.is_compromised:
            # Compromised/hacked nodes have a red outline around them
            node_edge_color_mask[i] = "#ff0000"
            name += " (compromised)"
        legend_map[name] = Patch(facecolor=node_color_mask[i],
                                      edgecolor=node_edge_color_mask[i])

    for j, (start_node, end_node) in enumerate(network.graph.edges()):
        # Edges / Communication Channels between 2 compromised nodes are compromised
        if start_node.is_compromised and end_node.is_compromised:
            edge_color_mask[j] = "#ff0000"

    # >> Plotting <<
    fig, axes = plt.subplots(nrows=1, ncols=2,  figsize=(24,6), width_ratios=[0.6, 0.4])
    label_map = {node:node.id for node in network.graph.nodes()}
    labels, handles = zip(*sorted(zip(*(legend_map.keys(), legend_map.values())), key=lambda t: t[0]))

    # Hierarchical / Tree Visualization of Communication Network
    tree_pos = hierarchy_pos(nx.to_undirected(network.graph), network.root)
    ax = axes[0]
    nx.draw_networkx_nodes(network.graph, pos=tree_pos, ax=ax,
                           node_size=400, node_shape="s", node_color=node_color_mask,
                           linewidths=1.0, edgecolors=node_edge_color_mask)
    nx.draw_networkx_labels(network.graph, pos=tree_pos, labels=label_map, ax=axes[0], font_size=10)
    nx.draw_networkx_edges(network.graph, pos=tree_pos, ax=axes[0], edge_color=edge_color_mask)
    
    
    # Spring Visualization of Communication Network
    spring_pos = nx.layout.spring_layout(network.graph)
    ax = axes[1]
    nx.draw_networkx_nodes(network.graph, pos=spring_pos, ax=ax,
                           node_size=400, node_shape="s", node_color=node_color_mask, 
                           linewidths=1.0, edgecolors=node_edge_color_mask, )
    nx.draw_networkx_labels(network.graph, pos=spring_pos, labels=label_map, ax=axes[1], font_size=10)
    nx.draw_networkx_edges(network.graph, pos=spring_pos, ax=axes[1], edge_color=edge_color_mask)
    # ax.legend(labels=labels, handles=handles)
    fig.legend(labels=labels, handles=handles, loc="lower center", bbox_to_anchor=(0.5, -0.1), ncol=len(labels),
               title="Legend", fancybox=True, fontsize='large', title_fontsize='larger')
    plt.tight_layout()
    plt.show()

plot_communication_network(pcn)

## Static Analysis
Given an infinite budget, breaksdown the probability of compromising components in the network. The resulting probabilities are exact (except for floating point precision issues) but do not scale well to larger communication networks (> 5 nodes). Useful as a static feature of a communication network. 

In [ ]:
import copy
from fractions import Fraction
# TODO: Account for probability of 0 devices being compromised

def iterate_over_paths(path, prob, reachable_nodes={}, visited_nodes={}, id_to_node={}):
    current_id = path[-1]
    current_node = id_to_node[current_id]
    visited_previously = current_id in visited_nodes
    if not visited_previously:
        visited_nodes[current_id] = None
    
    neighbouring_nodes = {k.id:None for k in current_node.get_neighbours()}
    reachable_nodes.update(neighbouring_nodes)
    reachable_nodes = {k:None for k in reachable_nodes if k not in visited_nodes}
    success_prob = current_node.get_prob_to_compromise()
    # If we fail, this path terminates
    yield path, prob*(1-success_prob), True
    if visited_previously:
        return
    n_reachable = len(reachable_nodes)
    reachable_ids =  list(reachable_nodes.keys())
    for reachable_node_id in reachable_ids:
        yield from iterate_over_paths(path+[reachable_node_id], prob*success_prob*(1/n_reachable),
                                      copy.copy(reachable_nodes), copy.copy(visited_nodes),
                                      id_to_node=id_to_node)
        
    # No more nodes reachable (entire network compromised)
    if len(reachable_nodes) == 0:
        yield path, prob*success_prob, False
def get_all_paths(graph):
    n_nodes = len(graph.nodes())
    id_to_node = {node.id:node for node in graph.nodes()}
    start_ids = list(id_to_node.keys())
    # Different starting locations
    for start_node_id in start_ids:
        yield from iterate_over_paths([start_node_id], prob=1/n_nodes,
                                      reachable_nodes={}, visited_nodes={},
                                      id_to_node=id_to_node)
     
sum_probs = 0.0
n_probs = {}
for path_no, (path, prob, ends_on_failure) in enumerate(get_all_paths(pcn.graph)):
    print(f"Path {path_no} :: Prob {str(Fraction(prob).limit_denominator()):<15} :: {'-'.join([str(node) for node in path])} :: {ends_on_failure}")
    if (len(path) > 1 and ends_on_failure) or (not ends_on_failure):
        path_length = len(path) - 1 if ends_on_failure else len(path)
        n_probs[path_length] = prob if path_length not in n_probs else n_probs[path_length] + prob
    sum_probs += prob
print(f"No. of Paths: {path_no}. Sum of Probabilities: {sum_probs} ({Fraction(sum_probs).limit_denominator()})")
n_probs[0] = sum(n_probs.values()) # TODO: Verify this
print("\n".join(f"{k} devices: {v}" for k,v in sorted(n_probs.items(),key=lambda item: item[0])))

In [ ]:
A = nx.adjacency_matrix(pcn.graph, nodelist=sorted(pcn.graph.nodes(), key=lambda node: node.id)).todense()
print("A\n", A)
np.fill_diagonal(A, val=0)
A2 = A@A
np.fill_diagonal(A2, val=0)
print("A^2\n", A2)
A3 = A2@A
np.fill_diagonal(A3, val=0)
print("A^3\n", A3)

In [ ]:
time_required = 0.0
nodes = pcn.graph.nodes()
node_probs = {}
for node in nodes:
    # print(node)
    probability_to_compromise = 1.0
    for defence_name, defence in node.defences.items():
        expected_effort = defence.effort_distribution.expect()
        time_required += expected_effort
        # print("\t", defence_name)
        probability_to_compromise *= defence.p
    node_probs[node] = probability_to_compromise

prob_to_compromise_n_devices = {}
all_nodes = set(nodes)
cumulative = 0.0
for n_devices in range(pcn.n_components, 0, -1):
    prob_to_compromise_n_devices[n_devices] = cumulative
    for combination in itertools.combinations(nodes, n_devices):
        probability_to_compromise = 1.0
        combination = set(combination)
        missing_nodes = all_nodes.difference(combination)
        for node in combination:
            probability_to_compromise *= node_probs[node]
        for node in missing_nodes:
            probability_to_compromise *= (1 - node_probs[node])
        prob_to_compromise_n_devices[n_devices] += probability_to_compromise 
    print(f"{n_devices} Devices: {prob_to_compromise_n_devices[n_devices]}")
    cumulative += prob_to_compromise_n_devices[n_devices]
print(prob_to_compromise_n_devices)

In [ ]:
# If the probability of compromising all components is the same,
# we can use the Binomial distribution function
# Takes: 12.6 µs
N = pcn.n_components
k = 2
p = 0.5
cumulative = 0.0
for k in range(pcn.n_components, 0, -1):
    prob = math.comb(N, k)*math.pow(p, k)*math.pow(1-p,N-k)
    print(f"{k} Devices: {cumulative + prob}")
    cumulative += prob

In [ ]:
import scipy.stats.distributions as distr
distr_lookup = {
    "TruncNorm": distr.truncnorm, # Continuous, loc=mean (float), scale=standard deviation (float)
    "Exponential": distr.expon, # Continuous, scale = 1 / lambda (float)
    "Gamma": distr.gamma, # Continuous, a = shape parameter (integer)
    "Bernoulli": distr.bernoulli, # Discrete
}
n_attacks = 20
is_successful = distr.bernoulli(0.5).rvs(size=n_attacks).astype(bool)
time_taken = distr.expon(scale=0.0).rvs(size=n_attacks)[is_successful]
print(f"Successful Attacks {sum(is_successful)}/{n_attacks}\nTime Taken per Successful Attack: {time_taken}")

## Communication Network Specifications
Explores how we can supply structured information to our procedural network generation algorithm. Includes information such as the types of components and defences we expect to see in the communication network.

In [ ]:
# seed = np.random.randint(low=0, high=52600)
seed = 27194
print(f"Seed: {seed}")
np.random.seed(seed)
pcn = CommNetwork(n_devices=15, n_entrypoints=1, children_per_parent=5, child_no_deviation=1,
                  network_specs="SmartMeterNetworkSpecifications.json",
                  enable_sibling_to_sibling_comm=True)
